In [0]:
# Inferential Statistics vs Big Data Analytics
## NYC Yellow Taxi Trips (2022–2024)

from pyspark.sql.functions import *
import pandas as pd
import matplotlib.pyplot as plt


### Charger le dataset SAMPLE
### 

In [0]:
sample_path = "/Volumes/workspace/default/filestore/yellowtaxisample1pct_hybrid_stratified.csv"

df_sample = pd.read_csv(sample_path)
df_sample.shape


In [0]:
df_sample.head()

In [0]:
df_sample.isnull().sum()

In [0]:
# 1/ passenger_count : remplir par la médiane
median_passenger = df_sample["passenger_count"].median()

df_sample["passenger_count"] = df_sample["passenger_count"].fillna(median_passenger)

# 2/RatecodeID : remplir par la valeur la plus fréquente (mode
mode_ratecode = df_sample["RatecodeID"].mode()[0]

df_sample["RatecodeID"] = df_sample["RatecodeID"].fillna(mode_ratecode)

# 3/ store_and_fwd_flag : remplir par "N"
df_sample["store_and_fwd_flag"] = df_sample["store_and_fwd_flag"].fillna("N")

# 4/ congestion_surcharge : remplir par 0
df_sample["congestion_surcharge"] = df_sample["congestion_surcharge"].fillna(0)

# 5/ Fusionner Airport_fee dans airport_fee
df_sample["airport_fee"] = (
    df_sample["airport_fee"]
    .fillna(df_sample["Airport_fee"])
    .fillna(0)
)

# Supprimer la colonne en double
df_sample = df_sample.drop(columns=["Airport_fee"])





In [0]:
df_sample.isnull().sum()


### Charger le dataset POPULATION

In [0]:
pop_path = "/Volumes/workspace/default/filestore/Data_as_paquets/"

df_pop = spark.read.parquet(pop_path)

df_pop.printSchema()
df_pop.count()



### **📌 Description des colonnes – NYC Yellow Taxi**
### 
**VendorID**
Identifiant du fournisseur de taxi **(ex : entreprise ou système de dispatch)**.

**tpep_pickup_datetime**
Date et heure de début de la course **(prise en charge du client)**.

**tpep_dropoff_datetime**
Date et heure de fin de la course.

**passenger_count**
Nombre de passagers dans le taxi.

**trip_distance**
Distance de la course en miles.

**RatecodeID**
Code tarifaire appliqué à la course **(tarif standard, aéroport, hors ville, etc.)**.

**store_and_fwd_flag**
Indique si les données de la course ont été stockées temporairement avant transmission
**(Y = oui, N = non)**.

**PULocationID**
Identifiant de la zone de prise en charge **(Pickup Location)**.

**DOLocationID**
Identifiant de la zone de dépose **(Dropoff Location)**.

**payment_type**
Méthode de paiement utilisée :

1 → Carte

2 → Cash

Autres → non standard

**fare_amount**
Montant de base de la course **(hors taxes et suppléments)**.

**extra**
Suppléments **(heures de nuit, heures de pointe, etc.)**.

**mta_tax**
Taxe MTA **(taxe fixe de transport à NYC)**.

**tip_amount**
Montant du pourboire.

**tolls_amount**
Frais de péage.

**improvement_surcharge**
Supplément réglementaire pour amélioration du service.

**congestion_surcharge**
Supplément lié à la congestion dans certaines zones.

**airport_fee**
Frais supplémentaires pour les trajets vers/depuis les aéroports.

**total_amount**
Montant total payé par le client **(tous frais inclus)**.

In [0]:
files = dbutils.fs.ls(pop_path)

df_list = []
for f in files:
    df_list.append(spark.read.parquet(f.path))

df_pop = df_list[0]
for df in df_list[1:]:
    df_pop = df_pop.unionByName(df, allowMissingColumns=True)

df_pop.count()


In [0]:
# Calcul des NULL
null_summary = df_pop.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_pop.columns
])

# Récupérer les résultats en dictionnaire
null_counts = null_summary.first().asDict()

# Garder uniquement les colonnes avec NULL
cols_with_nulls = [c for c, v in null_counts.items() if v > 0]

### les colonnes avec des NULL

In [0]:
null_summary.select(cols_with_nulls).show(truncate=False)
